In this notebook, we will be preparing anything that might be needed for future steps, such as co-expression networks and integrating different data sources (both from the challenge and from elsewhere).

### Co-Expression Network

--- WRITE SOME INTRO! --- # TODO

In [3]:
import os
os.getcwd()

'c:\\Users\\yayag\\Kaggle_SCP\\notebooks'

In [4]:
import dask.dataframe as dd
import pandas as pd

target_cell = "B cells"

# multiome_obs_meta = pd.read_csv("../data/multiome_obs_meta.csv")
# obs_ids = multiome_obs_meta[multiome_obs_meta["cell_type"] == target_cell]["obs_id"].to_list()

# multiome_var_meta = pd.read_csv("../data/multiome_var_meta.csv")
# gene_names = multiome_var_meta[multiome_var_meta["feature_type"] == "Gene Expression"]["location"].to_list()

adata_obs_meta = pd.read_csv("../data/adata_obs_meta.csv")
obs_ids = adata_obs_meta[adata_obs_meta["col"] == 3]["obs_id"].to_list()

In [11]:
adata_obs_meta[adata_obs_meta["col"] == 3][adata_obs_meta["plate_name"]=="plate_1"][adata_obs_meta["well"]=="A3"][adata_obs_meta["cell_type"]=="NK cells"]

C:\Users\yayag\AppData\Local\Temp\ipykernel_15668\2082445873.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  adata_obs_meta[adata_obs_meta["col"] == 3][adata_obs_meta["plate_name"]=="plate_1"][adata_obs_meta["well"]=="A3"][adata_obs_meta["cell_type"]=="NK cells"]
C:\Users\yayag\AppData\Local\Temp\ipykernel_15668\2082445873.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  adata_obs_meta[adata_obs_meta["col"] == 3][adata_obs_meta["plate_name"]=="plate_1"][adata_obs_meta["well"]=="A3"][adata_obs_meta["cell_type"]=="NK cells"]
C:\Users\yayag\AppData\Local\Temp\ipykernel_15668\2082445873.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  adata_obs_meta[adata_obs_meta["col"] == 3][adata_obs_meta["plate_name"]=="plate_1"][adata_obs_meta["well"]=="A3"][adata_obs_meta["cell_type"]=="NK cells"]


,obs_id,library_id,plate_name,well,row,col,cell_id,donor_id,cell_type,sm_lincs_id,sm_name,SMILES,dose_uM,timepoint_hr,control
448,00714301ff99b698,library_44,plate_1,A3,A,3,PBMC,donor_2,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
914,00ef8cd8be580bdc,library_44,plate_1,A3,A,3,PBMC,donor_2,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
2648,02be934dc2b2d2ee,library_44,plate_1,A3,A,3,PBMC,donor_2,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
3656,03c96af2582b0d1f,library_44,plate_1,A3,A,3,PBMC,donor_2,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
6616,0706c0880c35a96a,library_44,plate_1,A3,A,3,PBMC,donor_2,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233812,f9599f0cde859dfe,library_44,plate_1,A3,A,3,PBMC,donor_2,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
235846,fb75f6ca856f84fa,library_44,plate_1,A3,A,3,PBMC,donor_2,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
237328,fd13d1fc10720590,library_44,plate_1,A3,A,3,PBMC,donor_2,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
237751,fd860943f7f9de2f,library_44,plate_1,A3,A,3,PBMC,donor_2,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True


In [23]:
pd.DataFrame(adata_obs_meta.groupby(["plate_name", "well", "cell_type"])["obs_id"].apply(list)).to_csv("../config/obs_id_groups.csv")

In [6]:
from pyarrow.parquet import ParquetFile

target_adata_df = pd.DataFrame()
batch_num = 10

adata_file = ParquetFile("../data/adata_train.parquet")
batch_group_size = adata_file.metadata.num_rows // batch_num
for batch in adata_file.iter_batches(batch_size=batch_group_size):
    adata_batch = batch.to_pandas()
    adata_batch = adata_batch[adata_batch["obs_id"].isin(obs_ids)]
    target_adata_df = pd.concat([target_adata_df, adata_batch], axis=0, ignore_index=True)

combine observations gene expression for each cell type

In [28]:
exp_groups = pd.read_csv("../config/obs_id_groups.csv")
B_cells = exp_groups[exp_groups["cell_type"]=="B cells"]
Myeloid_cells = exp_groups[exp_groups["cell_type"]=="Myeloid cells"]
NK_cells = exp_groups[exp_groups["cell_type"]=="NK cells"]
T_cells = exp_groups[exp_groups["cell_type"]=="T cells CD4+"]
T_regulatory_cells = exp_groups[exp_groups["cell_type"]=="T regulatory cells"]

    

In [29]:
def create_combined_obs_exp(groups, exp_data):
    for row in groups["obs_id"]:
        sample = exp_data[exp_data["obs_id"].isin(row)]
        max_exp = sample.max()

,plate_name,well,cell_type,obs_id
0,plate_0,A1,B cells,"['00445d89e7501ef8', '034630e37dfa09ee', '0360..."
9,plate_0,A11,B cells,"['006ff4572ba5195b', '031a4ebff7ce1758', '0321..."
19,plate_0,A2,B cells,"['05d180f1ebd1f530', '0e0d311c44da054e', '0f3d..."
25,plate_0,A3,B cells,"['010facc6f76220ca', '035f4d18aa398e07', '08a4..."
35,plate_0,A5,B cells,"['01d06387631f3d2c', '01fa20c246425861', '0308..."
...,...,...,...,...
2469,plate_5,G2,B cells,"['0630ed27716f6b13', '06946c4963d71383', '0987..."
2475,plate_5,G3,B cells,"['08a6643b991938ba', '0ab1de634be3814b', '0cac..."
2505,plate_5,H1,B cells,"['002fba45c43e1cf3', '005b110cbf89d42d', '0405..."
2522,plate_5,H2,B cells,"['00486c33e4d3f880', '021a10f81757e361', '02c3..."


In [7]:
target_adata_df["obs_id"].unique().shape

(28462,)

In [8]:
target_data_df = target_adata_df.pivot_table("normalized_count", "obs_id", "gene")

In [31]:
target_data_df.max()

gene
A1BG        6.291455
A1BG-AS1    6.135656
A2M         7.143509
A2M-AS1     7.279147
A2MP1       6.541090
              ...   
ZXDC        6.600162
ZYG11A      5.662766
ZYG11B      6.799119
ZYX         6.886690
ZZEF1       6.920478
Length: 21122, dtype: float32

In [24]:
import PyWGCNA
exp = PyWGCNA.WGCNA(name=target_cell, species="Homo sapiens", geneExp=target_data_df.fillna(0))
exp.geneExpr.to_df().head(5)

gene,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
obs_id,,,,,,,,,,,,,,,,,,,,,
000683e13aa73f7f,0.0,0.0,0.0,5.73634,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
0008106bb52a931d,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
000a41a4580023f5,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,5.238042,0.0
000a59643eff306a,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,4.921865,...,0.000000,0.0,4.921865,0.0,0.0,0.000000,0.0,0.000000,5.611362,0.0
000a7dc7f1cba897,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,4.240418,...,4.240418,0.0,0.000000,0.0,0.0,4.240418,0.0,4.240418,4.240418,0.0


In [25]:
exp.preprocess()

Pre-processing...
	Detecting genes and samples with too many missing values...


In [ ]:
exp.geneExpr.to_df().head(5)

In [2]:
from pyarrow.parquet import ParquetFile

target_adata_df = pd.DataFrame()
batch_num = 10

adata_file = ParquetFile("../data/adata_train.parquet")
batch_group_size = adata_file.num_row_groups // batch_num
for batch in range(batch_num):
    if batch < batch_num - 1:
        s = range(batch*batch_group_size, (batch+1)*batch_group_size)
    else:
        s = range(batch*batch_group_size, adata_file.num_row_groups)
    adata_batch = adata_file.read_row_groups(s).to_pandas()
    adata_batch = adata_batch[adata_batch["obs_id"].isin(obs_ids)]
    target_adata_df = pd.concat([target_adata_df, adata_batch], axis=0, ignore_index=True)

In [23]:
target_data_df.notna().sum(axis=1)

obs_id
000683e13aa73f7f    1204
0008106bb52a931d    1052
000a41a4580023f5    1927
000a59643eff306a    1991
000a7dc7f1cba897    3580
                    ... 
fff6f64b0cd2160b    1609
fff97cc3be63ebc0    1355
fffb8bd514c27106    2382
fffc885a62e5dab7    2244
fffcb76d7d937647    4568
Length: 28462, dtype: int64

416442312

In [46]:
from pyarrow.parquet import ParquetFile

target_multiome_df = pd.DataFrame()
batch_num = 10

multiome_file = ParquetFile("../data/multiome_train.parquet")
batch_group_size = multiome_file.num_row_groups // batch_num
for batch in range(batch_num):
    if batch < batch_num - 1:
        s = range(batch*batch_group_size, (batch+1)*batch_group_size)
    else:
        s = range(batch*batch_group_size, multiome_file.num_row_groups)
    multiome_batch = multiome_file.read_row_groups(s).to_pandas()
    multiome_batch = multiome_batch[multiome_batch["obs_id"].isin(obs_ids)]
    # multiome_batch = multiome_batch[multiome_batch["location"].isin(gene_names)]
    target_multiome_df = pd.concat([target_multiome_df, multiome_batch], axis=0, ignore_index=True)

In [81]:
target_cell = "B cells"

adata_obs_meta = pd.read_csv("../data/adata_obs_meta.csv")
obs_ids = adata_obs_meta[adata_obs_meta["col"] == 3]["obs_id"].to_list()

In [88]:
x.fillna(0)

gene,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
obs_id,,,,,,,,,,,,,,,,,,,,,
000683e13aa73f7f,0.0,0.000000,0.000000,5.736340,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
0008106bb52a931d,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
000a41a4580023f5,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,5.238042,0.0
000a59643eff306a,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,4.921865,...,0.000000,0.000000,4.921865,0.0,0.0,0.000000,0.0,0.000000,5.611362,0.0
000a7dc7f1cba897,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,4.240418,...,4.240418,0.000000,0.000000,0.0,0.0,4.240418,0.0,4.240418,4.240418,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff6f64b0cd2160b,0.0,5.212115,0.000000,5.212115,0.0,0.0,0.000000,0.0,0.0,5.902534,...,0.000000,0.000000,0.000000,0.0,0.0,5.212115,0.0,0.000000,0.000000,0.0
fff97cc3be63ebc0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,6.239318,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
fffb8bd514c27106,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0


In [91]:
c = x[1:10].corr()
c

MemoryError: Unable to allocate 3.32 GiB for an array with shape (21122, 21122) and data type float64

In [53]:
# Transform dataframe --> columns as genes and rows as samples...
x = target_multiome_df[target_multiome_df["location"].isin(gene_names)].pivot_table("normalized_count", "obs_id", "location")

In [86]:
target_multiome_df

,obs_id,location,count,normalized_count
0,000225c1151ab841,AAMP,1,6.320659
1,000225c1151ab841,AASS,1,6.320659
2,000225c1151ab841,ABCC11,1,6.320659
3,000225c1151ab841,ABCC2,1,6.320659
4,000225c1151ab841,ABR,1,6.320659
...,...,...,...,...
34311166,ffe93b989e73959e,chrX:63754752-63755597,1,2.770587
34311167,ffe93b989e73959e,chrX:64205465-64206345,2,3.048007
34311168,ffe93b989e73959e,chrX:6738626-6739524,2,8.954666
34311169,ffe93b989e73959e,chrX:78754674-78755560,2,9.734200


In [70]:
(x["A1BG"].notna() & x["ZXDC"].notna()).sum()

51